In [715]:
import math

In [ ]:
sample_mapping = {
    "s1": "01",
    "s2": "02",
    "s3": "03",
    "s4": "04",
    "s5": "05",
    "s6": "06",
    "s7": "07",
    "s8": "08",
    "s9": "09",
    "s10": "10",
    "s11": "11",
    "s12", "12",
}

In [ ]:
sampling_round = 8
sample = 'xy'
sample_id = sample_mapping[sample]
file_name = ""
input_file = f''
session_type = "spk" if "spk" in input_file else "img"
output_file = f'/Project/data/cleaned_data/raw/{sample_id}_{session_type}_{int(math.ceil(sampling_round / 2))}.txt'

In [718]:
def remove_columns(input_file):
    columns_to_remove = ['CH9', 'CH19', 'CH27', 'CH30', 'CH61', 'ECG', 'VEP', 'EMG2']
    with open(input_file, 'r') as file:
        lines = file.readlines()
    
    column_names = lines[0].split()
    modified_lines = []
    counter = 0
    for line in lines:
        counter += 1
        columns = line.split()
        modified_columns = [col for i, col in enumerate(columns) if column_names[i] not in columns_to_remove]

        if counter % 10000 == 0:
            print(len(modified_columns), len(columns))

        modified_lines.append(modified_columns)
    return modified_lines

In [719]:
modified_lines = remove_columns(input_file)

57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65
57 65


In [720]:
correct_columns = ['F3',
 'F4',
 'P3',
 'AF3',
 'AFZ',
 'AF4',
 'P4',
 'F7',
 'F5',
 'F1',
 'FZ',
 'F2',
 'F6',
 'F8',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'FCZ',
 'FC2',
 'FC4',
 'FC6',
 'FT8',
 'T7',
 'C5',
 'C3',
 'C1',
 'CZ',
 'C2',
 'C4',
 'C6',
 'T8',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'CPZ',
 'CP2',
 'CP4',
 'CP6',
 'TP8',
 'P7',
 'P5',
 'P1',
 'PZ',
 'P2',
 'P6',
 'P8',
 'PO7',
 'PO3',
 'POZ',
 'PO4',
 'PO8',
 'O1',
 'OZ',
 'O2',
 'MARK']

In [721]:
modified_lines[0]

['F3',
 'F4',
 'P3',
 'AF3',
 'AFZ',
 'AF4',
 'P4',
 'F7',
 'F5',
 'F1',
 'FZ',
 'F2',
 'F6',
 'F8',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'FCZ',
 'FC2',
 'FC4',
 'FC6',
 'FT8',
 'T7',
 'C5',
 'C3',
 'C1',
 'CZ',
 'C2',
 'C4',
 'C6',
 'T8',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'CPZ',
 'CP2',
 'CP4',
 'CP6',
 'TP8',
 'P7',
 'P5',
 'P1',
 'PZ',
 'P2',
 'P6',
 'P8',
 'PO7',
 'PO3',
 'POZ',
 'PO4',
 'PO8',
 'O1',
 'OZ',
 'O2',
 'MARK']

In [722]:
assert correct_columns ==  modified_lines[0]

In [723]:
def print_triggers(lines):
    counter = 0
    triggers = []
    for index, line in enumerate(lines):
        if line[-1] != '0' and line[-1] != 'MARK':
            # print(index, line[-1])
            triggers.append((index, line[-1]))
            counter += 1
    print(f"Total number of trigges: {counter}")
    print(triggers)
    return triggers

In [724]:
def remove_trigger(lines, row_numbers):
    for line_number, line in enumerate(lines):
        if line_number in row_numbers:
            line[-1] = '0'

In [725]:
triggers = print_triggers(modified_lines)

Total number of trigges: 0
[]


In [726]:
# TODO To be commented
# 507967 - 225 * 2250

# remove_trigger(modified_lines, [triggers[0][0], triggers[1][0], triggers[3][0], triggers[5][0]])
# triggers = print_triggers(modified_lines)

In [727]:
assert len(triggers) == 9

AssertionError: 

In [ ]:
remove_trigger(modified_lines, [triggers[0][0], triggers[1][0], triggers[3][0], triggers[5][0], triggers[7][0]])
triggers = print_triggers(modified_lines)

In [ ]:
assert len(triggers) == 4

In [ ]:
import numpy as np

def get_noise(trigger_time):
    mean = 0
    scale = 60
    upper_bound = 100
    lower_bound = -upper_bound

    noise = np.random.normal(loc=mean, scale=scale, size=1)
    noise = np.clip(noise, lower_bound, upper_bound)
    return int(noise)

def get_marks(lines):
    counter = 0
    for line in lines:
        if line[-1] != '0':
            counter += 1
    return counter

def add_triggers(lines, from_row, until_tow, trigger_time):
    marks = get_marks(lines)
    if marks >= 15:
        raise Exception('Already modified')

    new_triggers_indices = []
    i = from_row
    while True:
        noise = get_noise(trigger_time)
        i += trigger_time 
        if i + trigger_time > until_tow:
            break
        new_triggers_indices.append(i + noise)

    for index in new_triggers_indices:
        lines[index][-1] = '3'

In [ ]:
# 19214 + 223 * 2250 + 2250

In [ ]:
# TODO to be commented
# add_triggers(lines=modified_lines, from_row=19214, until_tow=523214, trigger_time=2250)

In [ ]:
add_triggers(lines=modified_lines, from_row=triggers[2][0], until_tow=triggers[3][0], trigger_time=2250)

In [ ]:
triggers = print_triggers(modified_lines)

In [ ]:
def get_first_trigger_index(lines):
    for index, line in enumerate(lines):
        if line[-1] != '0' and line[-1] != 'MARK':
            return index

def get_last_trigger_index(lines):
    for index, line in enumerate(lines[::-1]):
        if line[-1] != '0' and line[-1] != 'MARK':
            return len(lines) - index

In [ ]:
def remove_lines_before_first_trigger_and_after_last_trigger(lines):
    threshold = 500
    first_index = get_first_trigger_index(lines)
    del lines[1: first_index - threshold]
    last_index = get_last_trigger_index(lines)
    del lines[last_index + 1 + threshold: len(lines)]
    return lines

In [ ]:
modified_lines = remove_lines_before_first_trigger_and_after_last_trigger(modified_lines)

In [ ]:
triggers = print_triggers(modified_lines)


In [ ]:
len(modified_lines)

In [ ]:
assert 225 * 2250 < len(modified_lines)

In [ ]:
assert len(triggers) == 226

In [ ]:
assert correct_columns ==  modified_lines[0]

In [ ]:
def save_lines_to_file(lines, output_file):
    with open(output_file, 'w') as file:
        for line in lines:
            file.write(' '.join(line) + '\n')

In [ ]:
modified_lines_without_header = modified_lines[1:]
save_lines_to_file(modified_lines_without_header, output_file)